In [ ]:
!pip install simpletransformers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
# !cd /kaggle/working

# !mkdir -p /kaggle/working/formatted_data/train

# !mkdir -p /kaggle/working/formatted_data/dev
# !mkdir -p /kaggle/working/formatted_data/test

# !ls /kaggle/working/formatted_data
# !rm -rf /kaggle/working/outputs
# !mkdir -p /kaggle/working/outputs

# !ls /kaggle/working/formatted_data/test
# !mv /kaggle/working/formatted_data/test/*_dev.tsv /kaggle/working/formatted_data/dev/ 
# !ls /kaggle/working/formatted_data/dev
# !ls /kaggle/working/formatted_data/test

In [ ]:
# splits = ['train', 'dev']
# languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo', 'twi', 'kr', 'ts', 'multilingual']

# for lang in languages:
#     for split in splits:
#         df = pd.read_csv(f"{INPUT_DIR}/{split}/{lang}_{split}.tsv", sep='\t')
               
#         df = df.drop(['ID'], axis = 1)
#         df.rename(columns={'tweet':'text', 'label':'labels'}, inplace=True)
# #         df.sample(5)
#         df.to_csv(f"{DATA_DIR}/{split}/{lang}_{split}.tsv", sep='\t', index=False)

# # !ls /kaggle/working/formatted_data/*
# df.sample(5)

In [ ]:
# !cp /kaggle/input/semevalafrisenti/comb_data/test/* /kaggle/working/formatted_data/test/
# !cp /kaggle/input/semevalafrisenti/comb_data/dev/* /kaggle/working/formatted_data/test/

# languages = ['_am_', 'dz', '_ha_', '_ig_', '_ma_', '_pcm_', 'pt', 'sw', '_yo_', '_twi', 'kr', 'ts', 'multilingual']

# kr_, _pt, _sw, dz, tshaa

lang = 'ts'


TRIAL = False
CLEAN = False
DEMOJI = False


DATA_DIR = '/kaggle/input/semevalafrisenti/comb_data'

WORKING_DIR = "/kaggle/temp/working_dir"




if not os.path.exists(WORKING_DIR):
    os.makedirs(WORKING_DIR)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import re
import emoji

def cleaner(tweet):
    whitespace = re.compile(r"\s+")
    web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
    user = re.compile(r"(?i)@[a-z0-9_]+")
    
#     tweet = text.replace("@user", "")
    tweet = whitespace.sub(' ', tweet)
    tweet = web_address.sub('', tweet)
    tweet = user.sub('', tweet)
    return tweet

In [ ]:
train_df = pd.read_csv(f"{DATA_DIR}/train/{lang}_train.tsv", sep = '\t')

train_df = train_df.drop(columns=['ID'])
train_df = train_df.rename(columns = {'tweet':'text', 'label':'labels'})

if TRIAL:
    train_df = train_df.sample(1000)

train_df["labels"] = [{
    'negative':0,
    'neutral':1,
    'positive':2,
}[v] for v in train_df["labels"]]

if DEMOJI:
    train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))    
if CLEAN:
    train_df['text'] = train_df['text'].apply(lambda x: cleaner(x))


train_df.sample(5)

In [ ]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 5

if TRIAL:
    model_args.num_train_epochs = 1
# model_args.labels_list = ["positive", "negative", "neutral"]
model_args.save_steps = -1
model_args.output_dir = WORKING_DIR
model_args.train_batch_size = 8
model_args.overwrite_output_dir = True
# model_args.best_model_dir = f"{OUTPUT_DIR}/best_model"


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

def get_scores(split, dev_true, dev_pred):
    print(len(dev_true), len(dev_pred))
    
    with open(f"{OUTPUT_DIR}/{lang}_{split}_scores.txt", 'w') as output_file:
        if split == 'dev':
            print(MODEL)
            print(classification_report(dev_true, dev_pred))
        output_file.write(classification_report(dev_true, dev_pred))

In [ ]:


# MODEL_LIST = ['afroxlmr', 'naijasenti', 'afriberta', 'xlmr', 'mbert']

MODEL_LIST = ['afroxlmr']

for MODEL in MODEL_LIST:
    

    VARIANT = ""
    # DATA_DIR = '/kaggle/working/formatted_data'
    OUTPUT_DIR = f"/kaggle/working/post_exp/{MODEL}{VARIANT}"
    
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
    
    
    if MODEL == 'afroxlmr':
        print("AFRO-XLMR")
        model = ClassificationModel(
            'xlmroberta', 'Davlan/afro-xlmr-base', args=model_args, num_labels=3
        )
    elif MODEL == 'naijasenti':
        print('NAIJASENTI')
        model = ClassificationModel(
            'xlmroberta', 'Davlan/naija-twitter-sentiment-afriberta-large', args=model_args, num_labels=3
        )
    elif MODEL == 'afriberta':
        print('AFRIBERTA')
        model = ClassificationModel(
            'xlmroberta', 'castorini/afriberta_base', args=model_args, num_labels=3
        )
    elif MODEL == 'xlmr':
        print('XLM-RoBERTa')
        model = ClassificationModel(
            'xlmroberta', 'xlm-roberta-base', args=model_args, num_labels=3
        )
    elif MODEL == 'mbert':
        print('MBERT')
        model = ClassificationModel(
            'bert', 'bert-base-multilingual-cased', args=model_args, num_labels=3
        )

    model.train_model(train_df)

    splits = ['train', 'dev', 'test']

    for split in splits:
        df = pd.read_csv(f"{DATA_DIR}/{split}/{lang}_{split}.tsv", sep = '\t')

        if TRIAL:
            df = df.sample(100)

        if split == 'train':
            df.rename(columns = {'text':'tweet', 'labels':'label'}, inplace=True)
        if DEMOJI:
            df['tweet'] = df['tweet'].apply(lambda x: emoji.demojize(x, delimiters=("", "")))    
        if CLEAN:
            df['tweet'] = df['tweet'].apply(lambda x: cleaner(x))

        pred, outputs = model.predict(list(df['tweet']))
        df['pred'] = pred
        df["pred"] = [{
            0:'negative',
            1:'neutral',
            2:'positive',
        }[v] for v in df["pred"]]

        if split != 'test':
            get_scores(split, list(df['label']), list(df['pred']))

        neg, neut, pos = [], [], []

        for output in outputs:
            neg.append(output[0])
            neut.append(output[1])
            pos.append(output[2])

        df['negative'] = neg
        df['neutral'] = neut
        df['positive'] = pos


        df.to_csv(f"{OUTPUT_DIR}/{lang}_{split}_output.tsv", sep='\t', index=False)
        


In [7]:
!ls fu


afriberta  afroxlmr  naijasenti


In [13]:
!cat full_outputs/afriberta/ig_dev_scores.txt

WEIGHTED F1

avg_precision:0.8033168738841202
avg_recall:0.8033677349266702
avg_f1_score:0.8033410742846602

MACRO F1

macro_precision:0.8025792478125137
macro_recall:0.8023123509445229
macro_f1_score:0.8024443091259535


In [ ]:
# !cat outputs/afriberta_er/train_scores.txt
# !ls outputs/afriberta

In [ ]:
# import pandas as pd

# df = pd.read_csv('outputs/afriberta/ha_train_output.tsv', sep = '\t')
# df